In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('AAPL_Train_Test.csv')

In [3]:
data

,DATE,IMPACT_SCORE,stock_pattern
0,2021-01-04,0.396424,0
1,2021-01-05,0.172300,0
2,2021-01-06,0.202093,1
3,2021-01-07,0.223581,1
4,2021-01-08,-0.288329,0
...,...,...,...
251,2022-01-03,0.461994,1
252,2022-01-04,0.547694,0
253,2022-01-05,0.059643,0
254,2022-01-06,0.001395,0


In [4]:
X,y = data['IMPACT_SCORE'].to_numpy(), data['stock_pattern'].astype(float).to_numpy()

In [5]:
X.max(), X.min()

(0.806269855, -0.7249699526666666)

In [6]:
X = torch.tensor(X, dtype=torch.float32).reshape(-1, 1)
y = torch.tensor(y, dtype=torch.float32)
X.shape, y.shape

(torch.Size([256, 1]), torch.Size([256]))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.layer1 = nn.Linear(1, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return self.sigmoid(x)

In [9]:
model_0 = BinaryClassificationModel()
criterion = nn.BCEWithLogitsLoss() # sigmoid activation function built in
# where ```nn.BCELoss() requieres inputs to have gone through the sigmoid activation function
optimizer = torch.optim.SGD(params=model_0.parameters(),
                           lr=0.1)
  # For binary classification
optimizer = optim.Adam(model_0.parameters(), lr=0.005)

In [10]:
def accuracy_(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [11]:
model_0.eval()
with torch.inference_mode():
    y_logits = model_0(X_test)[:5]
y_logits, y_test[:5]

(tensor([[0.5330],
         [0.5436],
         [0.5343],
         [0.5432],
         [0.5347]]),
 tensor([1., 1., 0., 0., 1.]))

In [12]:
y_pred_probs = torch.sigmoid(y_logits)
y_prob = torch.round(y_pred_probs.squeeze())
accuracy_(y_test[:5], torch.round(y_prob))

60.0

In [13]:
epochs = 100
for epoch in range(epochs):
    model_0.train()
    outputs = model_0(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(outputs))
    loss = criterion(outputs, y_train)
    acc = accuracy_(y_train, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")


Epoch 1/100, Loss: 0.7150048017501831
Epoch 11/100, Loss: 0.6950927376747131
Epoch 21/100, Loss: 0.6918361186981201
Epoch 31/100, Loss: 0.692146897315979
Epoch 41/100, Loss: 0.6918848752975464
Epoch 51/100, Loss: 0.6913567185401917
Epoch 61/100, Loss: 0.6910545229911804
Epoch 71/100, Loss: 0.6908134818077087
Epoch 81/100, Loss: 0.690589427947998
Epoch 91/100, Loss: 0.6903620958328247


In [14]:
model_0.eval()
with torch.inference_mode():
    test_logits = model_0(X_test).squeeze()
    A = (torch.sigmoid(test_logits) > 0.518).float()
    test_acc = accuracy_(y_test,
                           A)

test_acc

58.44155844155844

In [15]:
class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.layer_1 = nn.Linear(1, 300, bias=True) 
        self.layer_2 = nn.Linear(300, 100, bias=True)
        self.layer_out = nn.Linear(100, 1, bias=True) 
        
        
        self.relu = nn.ReLU()
        self.sigmoid =  nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.1, inplace=False)
        self.batchnorm1 = nn.BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batchnorm2 = nn.BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))
        
        return x

In [16]:
model_1 = ChurnModel()
criterion = nn.BCEWithLogitsLoss() # sigmoid activation function built in
# where ```nn.BCELoss() requieres inputs to have gone through the sigmoid activation function
  # For binary classification
optimizer = optim.Adam(model_1.parameters(), lr=0.001)

In [17]:
model_1.eval()
with torch.inference_mode():
    y_logits = model_1(X_test)[:5]
y_logits, y_test[:5]

(tensor([[0.4819],
         [0.4733],
         [0.4820],
         [0.4734],
         [0.4819]]),
 tensor([1., 1., 0., 0., 1.]))

In [18]:
y_pred_probs = torch.sigmoid(y_logits)
y_prob = torch.round(y_pred_probs.squeeze())
accuracy_(y_test[:5], torch.round(y_prob))

60.0

In [19]:
epochs = 100
for epoch in range(epochs):
    model_1.train()
    outputs = model_1(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(outputs))
    loss = criterion(outputs, y_train)
    acc = accuracy_(y_train, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

Epoch 1/100, Loss: 0.7152904272079468
Epoch 11/100, Loss: 0.6980375051498413
Epoch 21/100, Loss: 0.6901305913925171
Epoch 31/100, Loss: 0.6880898475646973
Epoch 41/100, Loss: 0.6831934452056885
Epoch 51/100, Loss: 0.6818185448646545
Epoch 61/100, Loss: 0.6771154999732971
Epoch 71/100, Loss: 0.6773819923400879
Epoch 81/100, Loss: 0.6747278571128845
Epoch 91/100, Loss: 0.6704778075218201


In [20]:
model_1.eval()
with torch.inference_mode():
    test_logits = model_1(X_test).squeeze()
    A = (torch.sigmoid(test_logits) > 0.58).float()
    test_acc = accuracy_(y_test,
                           A)

test_acc

58.44155844155844

In [21]:
# Amazon

data_AMZN = pd.read_csv('AMZN_Train_Test.csv')
X,y = data_AMZN['IMPACT_SCORE'].to_numpy(), data_AMZN['stock_pattern'].astype(float).to_numpy()
X = torch.tensor(X, dtype=torch.float32).reshape(-1, 1)
y = torch.tensor(y, dtype=torch.float32)
X.shape, y.shape
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(X, y, test_size=0.3, random_state=42)

In [22]:
model_0_AMZN = BinaryClassificationModel()
criterion = nn.BCEWithLogitsLoss() # sigmoid activation function built in
optimizer = optim.Adam(model_0_AMZN.parameters(), lr=0.005)

epochs = 100
for epoch in range(epochs):
    model_0_AMZN.train()
    outputs = model_0_AMZN(X_train_A).squeeze()
    y_pred = torch.round(torch.sigmoid(outputs))
    loss = criterion(outputs, y_train_A)
    acc = accuracy_(y_train_A, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

Epoch 1/100, Loss: 0.7518835663795471
Epoch 11/100, Loss: 0.7017771601676941
Epoch 21/100, Loss: 0.6937776207923889
Epoch 31/100, Loss: 0.6932641863822937
Epoch 41/100, Loss: 0.6931964755058289
Epoch 51/100, Loss: 0.6931796073913574
Epoch 61/100, Loss: 0.6931734681129456
Epoch 71/100, Loss: 0.6931703686714172
Epoch 81/100, Loss: 0.6931684613227844
Epoch 91/100, Loss: 0.693166971206665


In [23]:
model_0_AMZN.eval()
with torch.inference_mode():
    test_logits = model_0_AMZN(X_test_A).squeeze()
    A = (torch.sigmoid(test_logits) > 0.515).float()
    test_acc = accuracy_(y_test_A,
                           A)
test_acc

52.56410256410257

In [24]:
model_1_AMZN = ChurnModel()
criterion = nn.BCEWithLogitsLoss() # sigmoid activation function built in
# where ```nn.BCELoss() requieres inputs to have gone through the sigmoid activation function
  # For binary classification
optimizer = optim.Adam(model_1_AMZN.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model_1_AMZN.train()
    outputs = model_1_AMZN(X_train_A).squeeze()
    y_pred = torch.round(torch.sigmoid(outputs))
    loss = criterion(outputs, y_train_A)
    acc = accuracy_(y_train_A, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

Epoch 1/100, Loss: 0.7417848706245422
Epoch 11/100, Loss: 0.7327902913093567
Epoch 21/100, Loss: 0.729496955871582
Epoch 31/100, Loss: 0.7236664891242981
Epoch 41/100, Loss: 0.7195452451705933
Epoch 51/100, Loss: 0.7107111215591431
Epoch 61/100, Loss: 0.7038268446922302
Epoch 71/100, Loss: 0.7061998844146729
Epoch 81/100, Loss: 0.7008187174797058
Epoch 91/100, Loss: 0.6970139145851135


In [25]:
model_1_AMZN.eval()
with torch.inference_mode():
    test_logits = model_1_AMZN(X_test_A).squeeze()
    A = (torch.sigmoid(test_logits) > 0.63).float()
    test_acc = accuracy_(y_test_A,
                           A)

test_acc

56.41025641025641

In [26]:
# Microsoft

data_MSFT = pd.read_csv('MSFT_Train_Test.csv')
X,y = data_MSFT['IMPACT_SCORE'].to_numpy(), data_MSFT['stock_pattern'].astype(float).to_numpy()
X = torch.tensor(X, dtype=torch.float32).reshape(-1, 1)
y = torch.tensor(y, dtype=torch.float32)
X.shape, y.shape
X_train_M, X_test_M, y_train_M, y_test_M = train_test_split(X, y, test_size=0.3, random_state=42)

In [27]:
model_0_MSFT = BinaryClassificationModel()
criterion = nn.BCEWithLogitsLoss() # sigmoid activation function built in
optimizer = optim.Adam(model_0_MSFT.parameters(), lr=0.005)

epochs = 100
for epoch in range(epochs):
    model_0_MSFT.train()
    outputs = model_0_MSFT(X_train_M).squeeze()
    y_pred = torch.round(torch.sigmoid(outputs))
    loss = criterion(outputs, y_train_M)
    acc = accuracy_(y_train_M, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

Epoch 1/100, Loss: 0.6896273493766785
Epoch 11/100, Loss: 0.6812371611595154
Epoch 21/100, Loss: 0.6745429039001465
Epoch 31/100, Loss: 0.6695445775985718
Epoch 41/100, Loss: 0.6665372848510742
Epoch 51/100, Loss: 0.6638076305389404
Epoch 61/100, Loss: 0.661045491695404
Epoch 71/100, Loss: 0.6588720083236694
Epoch 81/100, Loss: 0.6574693322181702
Epoch 91/100, Loss: 0.6566143035888672


In [28]:
model_0_MSFT.eval()
with torch.inference_mode():
    test_logits = model_0_MSFT(X_test_M).squeeze()
    A = (torch.sigmoid(test_logits) > 0.35).float()
    test_acc = accuracy_(y_test_M,
                           A)
test_acc

58.2089552238806

In [29]:
model_1_MSFT = ChurnModel()
criterion = nn.BCEWithLogitsLoss() # sigmoid activation function built in
# where ```nn.BCELoss() requieres inputs to have gone through the sigmoid activation function
  # For binary classification
optimizer = optim.Adam(model_1_MSFT.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model_1_MSFT.train()
    outputs = model_1_MSFT(X_train_M).squeeze()
    y_pred = torch.round(torch.sigmoid(outputs))
    loss = criterion(outputs, y_train_M)
    acc = accuracy_(y_train_M, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

Epoch 1/100, Loss: 0.6826261281967163
Epoch 11/100, Loss: 0.6631788015365601
Epoch 21/100, Loss: 0.6573619842529297
Epoch 31/100, Loss: 0.6500556468963623
Epoch 41/100, Loss: 0.6480132341384888
Epoch 51/100, Loss: 0.6431382894515991
Epoch 61/100, Loss: 0.6386541724205017
Epoch 71/100, Loss: 0.6331751346588135
Epoch 81/100, Loss: 0.6309964656829834
Epoch 91/100, Loss: 0.6321460008621216


In [30]:
model_1_MSFT.eval()
with torch.inference_mode():
    test_logits = model_1_MSFT(X_test_M).squeeze()
    A = (torch.sigmoid(test_logits) > 0.515).float()
    test_acc = accuracy_(y_test_M,
                           A)

test_acc

46.26865671641791

In [31]:
# Microsoft

data_NFLX = pd.read_csv('NFLX_Train_Test.csv')
X,y = data_NFLX['IMPACT_SCORE'].to_numpy(), data_NFLX['stock_pattern'].astype(float).to_numpy()
X = torch.tensor(X, dtype=torch.float32).reshape(-1, 1)
y = torch.tensor(y, dtype=torch.float32)
X.shape, y.shape
X_train_N, X_test_N, y_train_N, y_test_N = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
model_0_NFLX = BinaryClassificationModel()
criterion = nn.BCEWithLogitsLoss() # sigmoid activation function built in
optimizer = optim.Adam(model_0_NFLX.parameters(), lr=0.005)

epochs = 100
for epoch in range(epochs):
    model_0_NFLX.train()
    outputs = model_0_NFLX(X_train_N).squeeze()
    y_pred = torch.round(torch.sigmoid(outputs))
    loss = criterion(outputs, y_train_N)
    acc = accuracy_(y_train_N, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

Epoch 1/100, Loss: 0.6941913366317749
Epoch 11/100, Loss: 0.6877966523170471
Epoch 21/100, Loss: 0.6870062351226807
Epoch 31/100, Loss: 0.6864122748374939
Epoch 41/100, Loss: 0.685556173324585
Epoch 51/100, Loss: 0.6848663091659546
Epoch 61/100, Loss: 0.6843106150627136
Epoch 71/100, Loss: 0.6837897896766663
Epoch 81/100, Loss: 0.6832754015922546
Epoch 91/100, Loss: 0.6827680468559265


In [33]:
model_0_NFLX.eval()
with torch.inference_mode():
    test_logits = model_0_NFLX(X_test_N).squeeze()
    A = (torch.sigmoid(test_logits) > 0.35).float()
    test_acc = accuracy_(y_test_N,
                           A)
test_acc

54.929577464788736

In [34]:
model_1_NFLX = ChurnModel()
criterion = nn.BCEWithLogitsLoss() # sigmoid activation function built in
# where ```nn.BCELoss() requieres inputs to have gone through the sigmoid activation function
  # For binary classification
optimizer = optim.Adam(model_1_NFLX.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model_1_NFLX.train()
    outputs = model_1_NFLX(X_train_N).squeeze()
    y_pred = torch.round(torch.sigmoid(outputs))
    loss = criterion(outputs, y_train_N)
    acc = accuracy_(y_train_N, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

Epoch 1/100, Loss: 0.6995649933815002
Epoch 11/100, Loss: 0.6867323517799377
Epoch 21/100, Loss: 0.6840085983276367
Epoch 31/100, Loss: 0.6773212552070618
Epoch 41/100, Loss: 0.670452356338501
Epoch 51/100, Loss: 0.6670268177986145
Epoch 61/100, Loss: 0.6664590835571289
Epoch 71/100, Loss: 0.6636513471603394
Epoch 81/100, Loss: 0.6620599031448364
Epoch 91/100, Loss: 0.6616595983505249


In [35]:
model_1_NFLX.eval()
with torch.inference_mode():
    test_logits = model_1_NFLX(X_test_N).squeeze()
    A = (torch.sigmoid(test_logits) > 0.555).float()
    test_acc = accuracy_(y_test_N,
                           A)

test_acc

54.929577464788736